# Cart-Pole


Install OpenAI Gym (https://gym.openai.com)

In [4]:
!pip install gym

Install the packages for visualizing Gym

In [5]:
!apt update
!apt install xvfb
!pip install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

## Random move

Import Gym

In [6]:
import gym

Import the packages for visualizing Gym

In [7]:
import base64
import io
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display

Move the cart-pole randomly

In [8]:
d = Display()
d.start()

env = Monitor(gym.make('CartPole-v0'),'./videos/random_move/', force=True)
print("env.observation_space.shape = ", env.observation_space.shape)
print("env.action_space.n = ", env.action_space.n)

obs = env.reset()

for t in range(100):
    obs, reward, is_done, info = env.step(env.action_space.sample())
    print("obs = ", obs)
    print("reward = ", reward)
    print("info = ", info)

    if is_done:
        print("Episode finished after {} timesteps".format(t+1))
        env.reset()
        break

for frame in env.videos:
    print("frame = ", frame)
    video = io.open(frame[0], 'r+b').read()
    encoded = base64.b64encode(video)

    display.display(display.HTML(data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))))

env.observation_space.shape =  (4,)
env.action_space.n =  2
obs =  [-0.01703366  0.23658968  0.01277255 -0.33578018]
reward =  1.0
info =  {}
obs =  [-0.01230186  0.04128831  0.00605694 -0.03909699]
reward =  1.0
info =  {}
obs =  [-0.0114761  -0.15391997  0.005275    0.25549079]
reward =  1.0
info =  {}
obs =  [-0.0145545  -0.34911684  0.01038482  0.54983286]
reward =  1.0
info =  {}
obs =  [-0.02153683 -0.54438311  0.02138148  0.84576956]
reward =  1.0
info =  {}
obs =  [-0.0324245  -0.73979016  0.03829687  1.1450988 ]
reward =  1.0
info =  {}
obs =  [-0.0472203  -0.93539084  0.06119884  1.44954122]
reward =  1.0
info =  {}
obs =  [-0.06592812 -0.74107219  0.09018967  1.17659015]
reward =  1.0
info =  {}
obs =  [-0.08074956 -0.93724251  0.11372147  1.49612949]
reward =  1.0
info =  {}
obs =  [-0.09949441 -0.74367204  0.14364406  1.24101138]
reward =  1.0
info =  {}
obs =  [-0.11436785 -0.94031589  0.16846429  1.57502556]
reward =  1.0
info =  {}
obs =  [-0.13317417 -0.74755524  0.199

## Q-learning

Import required packages

In [9]:
import numpy as np

Brain class

In [31]:
class Brain:
    def __init__(self, num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr):
        self.num_states = num_states
        self.list_state_range = list_state_range
        self.list_state_reso = list_state_reso
        self.num_actions = num_actions

        self.eps = 1.0  # for epsilon greedy algorithm
        self.gamma = gamma
        self.r = r
        self.lr = lr

        self.q_table = np.random.rand(np.prod(list_state_reso), num_actions)

    def bins(self, clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num + 1)[1:-1]
 
    def getStateIndex(self, observation):
        list_index = []
        for i in range(len(self.list_state_reso)):
            index = np.digitize(observation[i], bins=self.bins(self.list_state_range[i][0], self.list_state_range[i][1], self.list_state_reso[i]))
            list_index.append(index)
        return sum([index*int(np.prod(self.list_state_reso[:i])) for i, index in enumerate(list_index)])

    def updateQtable(self, obs, action, reward, next_obs):
        q = self.q_table[self.getStateIndex(obs), action]
        next_q_max = np.max(self.q_table[self.getStateIndex(next_obs)])
        self.q_table[self.getStateIndex(obs), action] = q + self.lr*(reward + self.gamma*next_q_max - q)

    def getAction(self, obs, is_training):
        if is_training and np.random.rand() < self.eps:
            action = np.random.randint(self.num_actions)
        else:
            action = np.argmax(self.q_table[self.getStateIndex(obs)])
        ## update eps
        if is_training and self.eps > 0.1:
            self.eps *= self.r
        return action

Agent class

In [28]:
class Agent:
    def __init__(self, num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr):
        self.brain = Brain(num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr)
 
    def updateQtable(self, obs, action, reward, next_obs):
        self.brain.updateQtable(obs, action, reward, next_obs)
 
    def getAction(self, obs, is_training):
        action = self.brain.getAction(obs, is_training)
        return action

Environment class

In [37]:
class Environment:
    def __init__(self, num_episodes, max_step, gamma, r, lr):
        ## parameters
        self.num_episodes = num_episodes
        self.max_step = max_step
        ## environment
        self.env = Monitor(gym.make('CartPole-v0'), './videos/q_learning/', force=True)
        ## agent
        num_states = self.env.observation_space.shape[0]    # position, velocity, angle, angular velocity
        list_state_range = []
        for i in range(num_states):
            list_state_range.append([env.observation_space.low[i], env.observation_space.high[i]])
        list_state_range[1] = [-3.0, 3.0]
        list_state_range[3] = [-0.5, 0.5]
        print("list_state_range = ", list_state_range)
        list_state_reso = [4, 4, 6, 6]
        num_actions = self.env.action_space.n

        self.agent = Agent(num_states, list_state_range, list_state_reso, num_actions, gamma, r, lr)
 
    def train(self):
        num_completed_episodes = 0
  
        for episode in range(self.num_episodes):
            obs = self.env.reset()
            episode_reward = 0
 
            for step in range(self.max_step):
                ## get action
                action = self.agent.getAction(obs, is_training=True)
                ## observe next step
                next_obs, _, is_done, _ = self.env.step(action)
                ## get reward
                if is_done:
                    if step < max_step - 1:
                        reward = -100
                    else:
                        reward = 1
                        num_completed_episodes += 1
                else:
                    reward = 1
                episode_reward += reward
                ## update
                self.agent.updateQtable(obs, action, reward, next_obs)
                ## to next step
                obs = next_obs

                if is_done:
                    print('{0} Episode: Finished after {1} time steps with reward {2}'.format(episode, step+1, episode_reward))
                    break
        print("num_completed_episodes = ", num_completed_episodes)

    def evaluate(self):
        obs = self.env.reset()
        
        for step in range(self.max_step):
            ## get action
            action = self.agent.getAction(obs, is_training=False)
            ## observe next step
            next_obs, _, is_done, _ = self.env.step(action)
            ## to next step
            obs = next_obs

            if is_done:
                print('Evaluation: Finished after {} time steps'.format(step+1))
                break

Prepare showing videos of the restults

In [26]:
def show_video(env):
    for frame in env.videos:
        print("frame = ", frame)
        video = io.open(frame[0], 'r+b').read()
        encoded = base64.b64encode(video)

        display.display(display.HTML(data="""
            <video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>
            """.format(encoded.decode('ascii')))
        )

Run training

In [38]:
## parameters
num_episodes = 500
max_step = 200
gamma = 0.9
r = 0.99
lr = 0.5

## run
cartpole_env = Environment(num_episodes, max_step, gamma, r, lr)
cartpole_env.train()
cartpole_env.evaluate()
show_video(cartpole_env.env)

list_state_range =  [[-4.8, 4.8], [-3.0, 3.0], [-0.41887903, 0.41887903], [-0.5, 0.5]]
0 Episode: Finished after 20 time steps with reward -81
1 Episode: Finished after 35 time steps with reward -66
2 Episode: Finished after 27 time steps with reward -74
3 Episode: Finished after 19 time steps with reward -82
4 Episode: Finished after 44 time steps with reward -57
5 Episode: Finished after 24 time steps with reward -77
6 Episode: Finished after 10 time steps with reward -91
7 Episode: Finished after 14 time steps with reward -87
8 Episode: Finished after 50 time steps with reward -51
9 Episode: Finished after 55 time steps with reward -46
10 Episode: Finished after 39 time steps with reward -62
11 Episode: Finished after 120 time steps with reward 19
12 Episode: Finished after 37 time steps with reward -64
13 Episode: Finished after 75 time steps with reward -26
14 Episode: Finished after 88 time steps with reward -13
15 Episode: Finished after 46 time steps with reward -55
16 Episode:

frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000001.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000001.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000008.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000008.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000027.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000027.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000064.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000064.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000125.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000125.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000216.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000216.meta.json')


frame =  ('/content/videos/q_learning/openaigym.video.11.60.video000343.mp4', '/content/videos/q_learning/openaigym.video.11.60.video000343.meta.json')


## DQN